## 1. Importing the Data
<p><img src="https://assets.datacamp.com/production/project_1230/img/nyc.jpg" alt="New York City skyline" width="600px">
<br>
Welcome to New York City (NYC), one of the most-visited cities in the world. 
As a result, there are many <a href="https://www.airbnb.com/"><em>Airbnb</em></a> listings to meet the high demand for temporary lodging for anywhere between a few nights to many months. 
In this notebook, we will take a look at the NYC Airbnb market by combining data from multiple file types like <code>.csv</code>, <code>.tsv</code>, and <code>.xlsx</code>.</p>
<p><br><br>
We will be working with three datasets:</p>
<ol>
<li><p><code>"datasets/airbnb_price.csv"</code></p></li>
<li><p><code>"datasets/airbnb_room_type.xlsx"</code></p></li>
<li><p><code>"datasets/airbnb_last_review.tsv"</code></p></li>
</ol>
<p><br><br>
Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:</p>
<ul>
<li>What is the average price, per night, of an Airbnb listing in NYC? </li>
<li>How does the average price of an Airbnb listing, per month, compare to the private rental market? </li>
<li>How many adverts are for private rooms? </li>
<li>How do Airbnb listing prices compare across the five NYC boroughs?</li>
</ul>

In [38]:
import pandas as pd
import numpy as np
import datetime as dt

prices=pd.read_csv('/Users/nguyenduc/PycharmProjects/DA-study/Project DA/data_cleaning/datasets/airbnb_price.csv')
xls=pd.ExcelFile('/Users/nguyenduc/PycharmProjects/DA-study/Project DA/data_cleaning/datasets/airbnb_room_type.xlsx')
room_types=xls.parse(0)
reviews=pd.read_csv('/Users/nguyenduc/PycharmProjects/DA-study/Project DA/data_cleaning/datasets/airbnb_last_review.tsv',sep='\t')

prices.head(),'\n',room_types.head(),'\n',reviews.head()

(   listing_id        price                nbhood_full
 0        2595  225 dollars         Manhattan, Midtown
 1        3831   89 dollars     Brooklyn, Clinton Hill
 2        5099  200 dollars     Manhattan, Murray Hill
 3        5178   79 dollars  Manhattan, Hell's Kitchen
 4        5238  150 dollars       Manhattan, Chinatown,
 '\n',
    listing_id                                description        room_type
 0        2595                      Skylit Midtown Castle  Entire home/apt
 1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
 2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
 3        5178            Large Furnished Room Near B'way     private room
 4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt,
 '\n',
    listing_id    host_name   last_review
 0        2595     Jennifer   May 21 2019
 1        3831  LisaRoxanne  July 05 2019
 2        5099        Chris  June 22 2019
 3        5178     Shunichi  June 24

## 2. Cleaning the price column
<p>Now the <code>DataFrames</code> have been loaded, the first step is to calculate the average price per listing by <code>room_type</code>. </p>
<p>You may have noticed that the <code>price</code> column in the <code>prices</code> DataFrame currently states each value as a string with the currency (dollars) following, i.e.,</p>
<pre><code>price
225 dollars
89 dollars
200 dollars</code></pre>
<p></p>
<p>We will need to clean the column in order to calculate the average price.</p>

In [39]:
# Remove whitespace and string characters from prices column
prices["price"] = prices["price"].str.replace(" dollars", "")

# Convert prices column to numeric datatype
prices["price"] = pd.to_numeric(prices['price'])

# Print descriptive statistics for the price column
prices["price"].describe()

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64

## 3. Calculating average price
<p>We can see three quarters of listings cost \$175 per night or less. </p>
<p>However, there are some outliers including a maximum price of \$7,500 per night! </p>
<p>Some of listings are actually showing as free. Let's remove these from the <code>DataFrame</code>, and calculate the average price.</p>

In [40]:
prices['price']

0        225
1         89
2        200
3         79
4        150
        ... 
25204    129
25205     45
25206    235
25207    100
25208     30
Name: price, Length: 25209, dtype: int64

In [41]:
#prices['price']=prices.loc(prices['price']>0)
#avg_price=round(prices['price'].mean(),2)
#avg_price

# Subset prices for listings costing $0, free_listings
free_listings = prices["price"] == 0

# Update prices by removing all free listings from prices
prices = prices.loc[~free_listings]

# Calculate the average price, avg_price
avg_price = round(prices["price"].mean(), 2)

# Print the average price
print("The average price per night for an Airbnb listing in NYC is ${}.".format(avg_price))

The average price per night for an Airbnb listing in NYC is $141.82.


## 4. Comparing costs to the private rental market
<p>Now we know how much a listing costs, on average, per night, but it would be useful to have a benchmark for comparison. 
According to <a href="https://www.zumper.com/rent-research">Zumper</a>, a 1 bedroom apartment in New York City costs, on average, $3,100 per month. Let's convert the per night prices of our listings into monthly costs, so we can compare to the private market. </p>

In [42]:
# Add a new column to the prices DataFrame, price_per_month
prices["price_per_month"] = prices["price"] * 365 / 12

# Calculate average_price_per_month
average_price_per_month = round(prices['price_per_month'].mean(), 2)

# Compare Airbnb and rental market
print("airbnb monthly costs are ${}, while in the private market you would pay {}.".format(average_price_per_month, "$3,100.00"))

airbnb monthly costs are $4313.61, while in the private market you would pay $3,100.00.


## 5. Cleaning the room type column
<p>Unsurprisingly, using Airbnb appears to be substantially more expensive than the private rental market. We should, however, consider that these Airbnb listings include single private rooms or even rooms to share, as well as entire homes/apartments. 
<br><br>
Let's dive deeper into the <code>room_type</code> column to find out the breakdown of listings by type of room. The <code>room_type</code> column has several variations for <code>private room</code> listings, specifically: </p>
<ul>
<li>"Private room"</li>
<li>"private room"</li>
<li>"PRIVATE ROOM"</li>
</ul>
<p>We can solve this by converting all string characters to lower case (upper case would also work just fine). </p>

In [43]:
room_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [44]:
# Convert the room_type column to lowercase

room_types["room_type"] = room_types["room_type"].str.lower()

# Update the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")

# Create the variable room_frequencies
room_frequencies = room_types['room_type'].value_counts()

# Print room_frequencies
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


## 6. What timeframe are we working with?
<p>It seems there is a fairly similar sized market opportunity for both private rooms (45% of listings) and entire homes/apartments (52%) on the Airbnb platform in NYC.
<br><br></p>
<p>Now let's turn our attention to the <code>reviews</code> DataFrame. The <code>last_review</code> column contains the date of the last review in the format of "Month Day Year" e.g., May 21 2019. We've been asked to find out the earliest and latest review dates in the DataFrame, and ensure the format allows this analysis to be easily conducted going forwards. </p>

In [45]:
# Change the data type of the last_review column to datetime
reviews["last_review"] = pd.to_datetime(reviews['last_review'])

# Create first_reviewed, the earliest review date
first_reviewed = reviews["last_review"].dt.date.min()

# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

# Print the oldest and newest reviews from the DataFrame
print("The latest Airbnb review is {}, the earliest review is {}".format(last_reviewed,first_reviewed))

The latest Airbnb review is 2019-07-09, the earliest review is 2019-01-01


## 7. Joining the DataFrames.
<p>Now we've extracted the information needed, we will merge the three DataFrames to make any future analysis easier to conduct. Once we have joined the data, we will remove any observations with missing values and check for duplicates.</p>

In [46]:
# Merge prices and room_types to create rooms_and_prices
rooms_and_prices = prices.merge(room_types, how='inner', on='listing_id')
#rooms_and_prices
# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews,how='inner',on='listing_id')

# Drop missing values from airbnb_merged
airbnb_merged=airbnb_merged.dropna()

# Check if there are any duplicate values
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


In [47]:
airbnb_merged

,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review
0,2595,225,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09
...,...,...,...,...,...,...,...,...
25197,36425863,129,"Manhattan, Upper East Side",3923.750000,Lovely Privet Bedroom with Privet Restroom,private room,Rusaa,2019-07-07
25198,36427429,45,"Queens, Flushing",1368.750000,No.2 with queen size bed,private room,H Ai,2019-07-07
25199,36438336,235,"Staten Island, Great Kills",7147.916667,Seas The Moment,private room,Ben,2019-07-07
25200,36442252,100,"Bronx, Mott Haven",3041.666667,1B-1B apartment near by Metro,entire home/apt,Blaine,2019-07-07


## 8. Analyzing listing prices by NYC borough
<p>Now we have combined all data into a single DataFrame, we will turn our attention to understanding the difference in listing prices between <a href="https://en.wikipedia.org/wiki/Boroughs_of_New_York_City">New York City boroughs</a>. 
We can currently see boroughs listed as the first part of a string within the <code>nbhood_full</code> column, e.g., </p>
<pre><code>Manhattan, Midtown
Brooklyn, Clinton Hill
Manhattan, Murray Hill
Manhattan, Hell's Kitchen
Manhattan, Chinatown</code></pre>
<p></p>
<p>We will therefore need to extract this information from the string and store in a new column, <code>borough</code>, for analysis.</p>

In [48]:
airbnb_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25184 entries, 0 to 25201
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   listing_id       25184 non-null  int64         
 1   price            25184 non-null  int64         
 2   nbhood_full      25184 non-null  object        
 3   price_per_month  25184 non-null  float64       
 4   description      25184 non-null  object        
 5   room_type        25184 non-null  category      
 6   host_name        25184 non-null  object        
 7   last_review      25184 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 1.6+ MB


In [49]:
# Extract information from the nbhood_full column and store as a new column,
airbnb_merged["borough"] = airbnb_merged['nbhood_full'].str.partition(",")[0]
# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby('borough')['price'].agg(['sum','mean','median','count'])

# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values('mean',ascending=False)
# Print boroughs
boroughs

,sum,mean,median,count
borough,,,,
Manhattan,1898417,184.04,149.0,10315
Brooklyn,1275250,122.02,95.0,10451
Queens,320715,92.83,70.0,3455
Staten Island,22974,86.04,71.0,267
Bronx,55156,79.25,65.0,696


## 9. Price range by borough
<p>The above output gives us a summary of prices for listings across the 5 boroughs. In this final task we would like to categorize listings based on whether they fall into specific price ranges, and view this by borough. 
<br><br>
We can do this using percentiles and labels to create a new column, <code>price_range</code>, in the DataFrame.
Once we have created the labels, we can then group the data and count frequencies for listings in each price range by borough.
<br><br>
We will assign the following categories and price ranges:</p>
<table>
<thead>
<tr>
<th>label</th>
<th>price</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>Budget</code></td>
<td>\$0-69</td>
</tr>
<tr>
<td><code>Average</code></td>
<td>\$70-175</td>
</tr>
<tr>
<td><code>Expensive</code></td>
<td>\$176-350</td>
</tr>
<tr>
<td><code>Extravagant</code></td>
<td>&gt; \$350</td>
</tr>
</tbody>
</table>

In [50]:
# Create labels for the price range, label_names
label_names = ['Budget','Average','Expensive','Extravagant']
# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]

# Insert new column, price_range, into DataFrame
airbnb_merged["price_range"] = pd.cut(airbnb_merged['price'], bins=ranges, labels=label_names)

# Calculate borough and price_range frequencies, prices_by_borough
prices_by_borough = airbnb_merged.groupby(['borough','price_range']).count()

prices_by_borough

listing_id  price  nbhood_full  price_per_month  \
borough       price_range                                                    
Bronx         Budget              381    381          381              381   
              Average             285    285          285              285   
              Expensive            25     25           25               25   
              Extravagant           5      5            5                5   
Brooklyn      Budget             3194   3194         3194             3194   
              Average            5532   5532         5532             5532   
              Expensive          1466   1466         1466             1466   
              Extravagant         259    259          259              259   
Manhattan     Budget             1148   1148         1148             1148   
              Average            5285   5285         5285             5285   
              Expensive          3072   3072         3072             3072   
              Extravagant         810    810          810              810   
Queens        Budget             1631   1631         1631             1631   
              Average            1505   1505         1505             1505   
              Expensive           291    291          291              291   
              Extravagant          28     28           28               28   
Staten Island Budget              124    124          124              124   
              Average             123    123          123              123   
              Expensive            20     20           20               20   
              Extravagant           0      0            0                0   

                           description  room_type  host_name  last_review  
borough       price_range                                                  
Bronx         Budget               381        381        381          381  
              Average              285        285        285          285  
              Expensive             25         25         25           25  
              Extravagant            5          5          5            5  
Brooklyn      Budget              3194       3194       3194         3194  
              Average             5532       5532       5532         5532  
              Expensive           1466       1466       1466         1466  
              Extravagant          259        259        259          259  
Manhattan     Budget              1148       1148       1148         1148  
              Average             5285       5285       5285         5285  
              Expensive           3072       3072       3072         3072  
              Extravagant          810        810        810          810  
Queens        Budget              1631       1631       1631         1631  
              Average             1505       1505       1505         1505  
              Expensive            291        291        291          291  
              Extravagant           28         28         28           28  
Staten Island Budget               124        124        124          124  
              Average              123        123        123          123  
              Expensive             20         20         20           20  
              Extravagant            0          0          0            0